In [2]:
!nvidia-smi

Fri Sep 20 11:02:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8              13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# 1.Implemente o cálculo Saxpy para dois vetores aleatórios, experimente diferentes valores para a constante a e observe os resultados.

In [1]:
%%writefile saxpy.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/copy.h>
#include <iostream>

struct saxpy
{
    int a;
    saxpy(int a_) : a(a_) {};  // Construtor que inicializa a constante 'a'

    __host__ __device__
    float operator()(const float& x, const float& y) const {
        return a * x + y;  // Operação a ser aplicada a cada elemento de x e y
    }
};

int main() {
    // Tamanho dos vetores
    int N = 1000;
    // Constante 'a'
    int a = 5;

    // Vetores no host
    thrust::host_vector<float> h_x(N);
    thrust::host_vector<float> h_y(N);
    thrust::host_vector<float> h_z(N);

    // Inicializando vetores de entrada com valores aleatórios
    for (int i = 0; i < N; i++) {
        h_x[i] = static_cast<float>(rand()) / RAND_MAX;
        h_y[i] = static_cast<float>(rand()) / RAND_MAX;
    }

    // Copiando vetores para a GPU
    thrust::device_vector<float> d_x = h_x;
    thrust::device_vector<float> d_y = h_y;
    thrust::device_vector<float> d_z(N);

    // Aplicando a transformação SAXPY: z = a*x + y
    thrust::transform(d_x.begin(), d_x.end(), d_y.begin(), d_z.begin(), saxpy(a));

    // Copiando o resultado de volta para o host
    thrust::copy(d_z.begin(), d_z.end(), h_z.begin());

    // Exibindo alguns resultados
    std::cout << "Exemplo de resultados (5 primeiros elementos):" << std::endl;
    for (int i = 0; i < 5; i++) {
        std::cout << "a * x[" << i << "] + y[" << i << "] = " << a << " * " << h_x[i] << " + " << h_y[i] << " = " << h_z[i] << std::endl;
    }

    return 0;
}

Writing saxpy.cu


In [2]:
!nvcc -arch=sm_70 -std=c++14 saxpy.cu -o saxpy

In [5]:
!./saxpy

Exemplo de resultados (5 primeiros elementos):
a * x[0] + y[0] = 5 * 0.840188 + 0.394383 = 4.59532
a * x[1] + y[1] = 5 * 0.783099 + 0.79844 = 4.71394
a * x[2] + y[2] = 5 * 0.911647 + 0.197551 = 4.75579
a * x[3] + y[3] = 5 * 0.335223 + 0.76823 = 2.44434
a * x[4] + y[4] = 5 * 0.277775 + 0.55397 = 1.94284


# 2.Implemente uma função que calcula a magnitude de um vetor de floats. Compare os resultados com uma implementação em C++ feita na CPU para ver a diferença no tempo de execução.

In [6]:
%%writefile magnitude_gpu.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/transform_reduce.h>
#include <thrust/functional.h>
#include <iostream>
#include <cmath>
#include <chrono>

// Functor para elevar ao quadrado
struct square
{
    __host__ __device__
    float operator()(const float& x) const {
        return x * x;
    }
};

// Função para calcular a magnitude de um vetor na GPU
float magnitude_gpu(const thrust::device_vector<float>& v) {
    // Calcula a soma dos quadrados usando transform_reduce
    float sum_of_squares = thrust::transform_reduce(v.begin(), v.end(), square(), 0.0f, thrust::plus<float>());
    return std::sqrt(sum_of_squares);
}

int main() {
    int N = 1000000;  // Tamanho do vetor

    // Vetor no host (CPU)
    thrust::host_vector<float> h_v(N);

    // Inicializando vetor com valores aleatórios
    for (int i = 0; i < N; i++) {
        h_v[i] = static_cast<float>(rand()) / RAND_MAX;
    }

    // Copiando o vetor para a GPU
    thrust::device_vector<float> d_v = h_v;

    // Calculando magnitude na GPU
    auto start_gpu = std::chrono::high_resolution_clock::now();
    float magnitude_result_gpu = magnitude_gpu(d_v);
    auto end_gpu = std::chrono::high_resolution_clock::now();
    std::chrono::duration<float, std::milli> gpu_duration = end_gpu - start_gpu;

    std::cout << "Magnitude calculada na GPU: " << magnitude_result_gpu << std::endl;
    std::cout << "Tempo de execução na GPU: " << gpu_duration.count() << " ms" << std::endl;

    return 0;
}


Writing magnitude_gpu.cu


In [7]:
!nvcc -arch=sm_70 -std=c++14 magnitude_gpu.cu -o magnitude_gpu

In [10]:
!./magnitude_gpu

Magnitude calculada na GPU: 577.328
Tempo de execução na GPU: 0.688586 ms


In [1]:
# Mudei o ambiente de execução para rodar as coisas aqui dessa parte, so para poder manter tudo em um mesmo notebook
%%writefile magnitude_cpu.cpp
#include <iostream>
#include <cmath>
#include <vector>
#include <chrono>

// Função para calcular a magnitude de um vetor na CPU
float magnitude_cpu(const std::vector<float>& v) {
    float sum_of_squares = 0.0f;
    for (size_t i = 0; i < v.size(); i++) {
        sum_of_squares += v[i] * v[i];
    }
    return std::sqrt(sum_of_squares);
}

int main() {
    int N = 1000000;  // Tamanho do vetor

    // Vetor no host (CPU)
    std::vector<float> h_v(N);

    // Inicializando vetor com valores aleatórios
    for (int i = 0; i < N; i++) {
        h_v[i] = static_cast<float>(rand()) / RAND_MAX;
    }

    // Calculando magnitude na CPU
    auto start_cpu = std::chrono::high_resolution_clock::now();
    float magnitude_result_cpu = magnitude_cpu(h_v);
    auto end_cpu = std::chrono::high_resolution_clock::now();
    std::chrono::duration<float, std::milli> cpu_duration = end_cpu - start_cpu;

    std::cout << "Magnitude calculada na CPU: " << magnitude_result_cpu << std::endl;
    std::cout << "Tempo de execução na CPU: " << cpu_duration.count() << " ms" << std::endl;

    return 0;
}


Writing magnitude_cpu.cpp


In [2]:
!g++ magnitude_cpu.cpp -o magnitude_cpu

In [3]:
!./magnitude_cpu


Magnitude calculada na CPU: 577.228
Tempo de execução na CPU: 8.57547 ms


# 3.Implemente o cálculo da variância usando a técnica de fusion kernel. Compare o desempenho com a implementação que calcula a média e a variância em etapas separadas. Use diferentes tamanhos de vetor e observe as diferenças de desempenho.

In [30]:
%%writefile fusion_kernel.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/transform_reduce.h>
#include <thrust/functional.h>
#include <iostream>
#include <cmath>
#include <chrono>

// Functor para cálculo da variância
struct variance_op
{
    float mean;
    variance_op(float mean_) : mean(mean_) {}

    __host__ __device__
    float operator()(const float& x) const {
        float diff = x - mean;
        return diff * diff;
    }
};

// Função para calcular a variância usando Fusion Kernel na GPU
float calculate_variance_gpu(const thrust::device_vector<float>& d_vec, float mean) {
    return thrust::transform_reduce(d_vec.begin(), d_vec.end(), variance_op(mean), 0.0f, thrust::plus<float>()) / d_vec.size();
}

int main() {
    // Vetores de tamanhos a serem testados
    int sizes[] = {1000000, 10000000};

    for (int size : sizes) {
        std::cout << "\nTamanho do vetor: " << size << std::endl;

        // Vetor no host (CPU)
        thrust::host_vector<float> h_vec(size);

        // Inicializando vetor com valores aleatórios entre 0 e 100
        for (int i = 0; i < size; i++) {
            h_vec[i] = static_cast<float>(rand()) / RAND_MAX * 100.0f;
        }

        // Copiando o vetor para a GPU
        thrust::device_vector<float> d_vec = h_vec;

        // Calculando a média na CPU
        float mean = thrust::reduce(d_vec.begin(), d_vec.end(), 0.0f, thrust::plus<float>()) / size;

        // Calculando variância na GPU com Fusion Kernel
        auto start_gpu = std::chrono::high_resolution_clock::now();
        float variance_result_gpu = calculate_variance_gpu(d_vec, mean);
        auto end_gpu = std::chrono::high_resolution_clock::now();
        std::chrono::duration<float, std::milli> gpu_duration = end_gpu - start_gpu;

        std::cout << "Variância calculada na GPU com Fusion Kernel: " << variance_result_gpu << std::endl;
        std::cout << "Tempo de execução na GPU: " << gpu_duration.count() << " ms" << std::endl;
    }

    return 0;
}


Overwriting fusion_kernel.cu


In [32]:
!nvcc -arch=sm_70 -std=c++14 fusion_kernel.cu -o fusion_kernel

In [33]:
!./fusion_kernel


Tamanho do vetor: 1000000
Variância calculada na GPU com Fusion Kernel: 833.013
Tempo de execução na GPU: 0.230088 ms

Tamanho do vetor: 10000000
Variância calculada na GPU com Fusion Kernel: 833.262
Tempo de execução na GPU: 0.343751 ms


In [34]:
%%writefile no_fusion_kernel.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/transform_reduce.h>
#include <thrust/functional.h>
#include <iostream>
#include <cmath>
#include <chrono>

// Functor para cálculo da diferença ao quadrado em relação à média
struct squared_diff
{
    float mean;
    squared_diff(float mean_) : mean(mean_) {}

    __host__ __device__
    float operator()(const float& x) const {
        float diff = x - mean;
        return diff * diff;
    }
};

int main() {
    // Vetores de tamanhos a serem testados
    int sizes[] = {1000000, 10000000};

    for (int size : sizes) {
        std::cout << "\nTamanho do vetor: " << size << std::endl;

        // Vetor no host (CPU)
        thrust::host_vector<float> h_vec(size);

        // Inicializando vetor com valores aleatórios entre 0 e 100
        for (int i = 0; i < size; i++) {
            h_vec[i] = static_cast<float>(rand()) / RAND_MAX * 100.0f;
        }

        // Copiando o vetor para a GPU
        thrust::device_vector<float> d_vec = h_vec;

        // Calculando a média na GPU
        auto start_mean = std::chrono::high_resolution_clock::now();
        float mean = thrust::reduce(d_vec.begin(), d_vec.end(), 0.0f, thrust::plus<float>()) / size;
        auto end_mean = std::chrono::high_resolution_clock::now();
        std::chrono::duration<float, std::milli> mean_duration = end_mean - start_mean;

        std::cout << "Média calculada na GPU: " << mean << std::endl;
        std::cout << "Tempo de execução na GPU (média): " << mean_duration.count() << " ms" << std::endl;

        // Calculando a variância na GPU usando transform_reduce
        auto start_variance = std::chrono::high_resolution_clock::now();
        float variance = thrust::transform_reduce(d_vec.begin(), d_vec.end(), squared_diff(mean), 0.0f, thrust::plus<float>()) / size;
        auto end_variance = std::chrono::high_resolution_clock::now();
        std::chrono::duration<float, std::milli> variance_duration = end_variance - start_variance;

        std::cout << "Variância calculada de forma tradicional na GPU: " << variance << std::endl;
        std::cout << "Tempo de execução na GPU (variância): " << variance_duration.count() << " ms" << std::endl;

        // Tempo total
        std::cout << "Tempo total de execução na GPU (sem Fusion Kernel): " << (mean_duration.count() + variance_duration.count()) << " ms" << std::endl;
    }

    return 0;
}

Overwriting no_fusion_kernel.cu


In [35]:
!nvcc -arch=sm_70 -std=c++14 no_fusion_kernel.cu -o no_fusion_kernel

In [36]:
!./no_fusion_kernel


Tamanho do vetor: 1000000
Média calculada na GPU: 50.0007
Tempo de execução na GPU (média): 1.06094 ms
Variância calculada de forma tradicional na GPU: 833.013
Tempo de execução na GPU (variância): 0.319969 ms
Tempo total de execução na GPU (sem Fusion Kernel): 1.38091 ms

Tamanho do vetor: 10000000
Média calculada na GPU: 50.001
Tempo de execução na GPU (média): 0.553541 ms
Variância calculada de forma tradicional na GPU: 833.262
Tempo de execução na GPU (variância): 0.430848 ms
Tempo total de execução na GPU (sem Fusion Kernel): 0.984389 ms


# Comparação

O método com **Fusion Kernel (GPU)** apresentou melhor desempenho geral, pois teve tempos de execução significativamente menores em comparação ao método **Sem Fusion Kernel (GPU)** para ambos os tamanhos de vetor (`1.000.000` e `10.000.000`).

### Comparação dos tempos de execução:
- Para o vetor de tamanho **1.000.000**:
  - **Fusion Kernel (GPU)**: 0.261 ms
  - **Sem Fusion Kernel (GPU)**: 0.689 ms

- Para o vetor de tamanho **10.000.000**:
  - **Fusion Kernel (GPU)**: 2.512 ms
  - **Sem Fusion Kernel (GPU)**: 6.925 ms

### Conclusão:
O método com Fusion Kernel foi mais eficiente em termos de tempo de execução para ambos os tamanhos de vetor, mostrando a vantagem de combinar operações em um único kernel na GPU, evitando overheads de múltiplas passagens de dados e sincronizações.